<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/SR_WhisperAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 音声認識デモ

## ライブラリのインストール

In [1]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.9 MB/s eta 0:00:00


## ライブラリのインポート

In [2]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
import openai
openai.api_key = ""#@param

### 準備

In [3]:
RECORD = """
  const sleep = time => new Promise(resolve => setTimeout(resolve, time))
  const b2text = blob => new Promise(resolve => {
    const reader = new FileReader()
    reader.onloadend = e => resolve(e.srcElement.result)
    reader.readAsDataURL(blob)
  })
  var record = time => new Promise(async resolve => {
    stream = await navigator.mediaDevices.getUserMedia({ audio: true })
    recorder = new MediaRecorder(stream)
    chunks = []
    recorder.ondataavailable = e => chunks.push(e.data)
    recorder.start()
    await sleep(time)
    recorder.onstop = async ()=>{
      blob = new Blob(chunks)
      text = await b2text(blob)
      resolve(text)
    }
    recorder.stop()
  })
"""

### 音声認識用関数

In [4]:
def speech_to_text(model='whisper-1', language='ja', second=5):
  filename='tmp.wav'
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (second * 1000))
  b = b64decode(s.split(',')[1])

  with open(filename, 'wb+') as fw:
    fw.write(b)

  with open(filename, "rb") as fr:
    transcription = openai.Audio.transcribe(
        model=model, 
        file=fr, 
        language=language
    )
    return transcription['text']

## 認識させてみましょう
実行したらマイクに向かって喋ってください。

In [5]:
print("認識スタート（５秒）")
text = speech_to_text()
print("認識結果：", text)
print("認識終了")

認識スタート（５秒）


<IPython.core.display.Javascript object>

認識結果： こんにちは 喋っています よろしくお願いします
認識終了
